In [ ]:
https://github.com/freedoom/attic

In [1]:
// Clone the repository to avoid rate limits and increase speed of parsing
const cloneCmd = new Deno.Command("git", {
  args: [
    "clone",
    "--bare",
    "https://github.com/freedoom/attic",
    "attic.git",
  ],
});

cloneCmd.outputSync();


{
  success: true,
  code: 0,
  signal: null,
  stdout: [Getter],
  stderr: [Getter]
}

In [ ]:
import { TextLineStream } from "jsr:@std/streams"; // specific import for Deno

const CONFIG = {
  repoPath: "attic.git",
  githubUrl: "https://github.com/freedoom/attic",
  // Match any file containing POSS (case insensitive)
  fileRegex: /POSS/i,
  outputFile: "scan_results-attic.json",
};

/**
 * Helper: Runs git ls-tree to get files in a SPECIFIC folder at a SPECIFIC commit.
 * This finds "Existing" files only in that folder.
 */
async function getSnapshotFiles(
  sha: string,
  folderPath: string,
): Promise<string[]> {
  const cmd = new Deno.Command("git", {
    args: [
      "--git-dir",
      CONFIG.repoPath,
      "ls-tree",
      "-r", // Recursive (relative to the path passed below)
      "--name-only", // Just filenames
      "--full-name", // Print full path relative to repo root
      sha, // Commit hash
      folderPath, // SCOPE: Only look inside this specific folder
    ],
    stdout: "piped",
  });

  const { stdout } = await cmd.output();
  const output = new TextDecoder().decode(stdout);

  return output
    .split("\n")
    .map((s) => s.trim())
    // 1. Must be valid length
    // 2. Must match POSS regex
    // 3. Must actually be inside the folder (double check to prevent partial matches)
    .filter((s) =>
      s.length > 0 && CONFIG.fileRegex.test(s) && s.startsWith(folderPath)
    );
}

function getDirectory(path: string): string {
  const lastSlashIndex = path.lastIndexOf("/");
  if (lastSlashIndex === -1) return "."; // Root
  return path.substring(0, lastSlashIndex);
}

async function main() {
  console.log(`📂 Opening Repo: ${CONFIG.repoPath}`);

  try {
    await Deno.stat(CONFIG.repoPath);
  } catch {
    console.error(`❌ Error: Path not found: ${CONFIG.repoPath}`);
    Deno.exit(1);
  }

  const cmd = new Deno.Command("git", {
    args: [
      "--git-dir",
      CONFIG.repoPath,
      "log",
      "--name-status",
      "--pretty=format:__COMMIT__|%H|%cd|%an|%s",
      "--date=iso-strict",
      "HEAD",
    ],
    stdout: "piped",
  });

  const process = cmd.spawn();
  console.log("🚀 Git process started. Scanning history...");

  const lines = process.stdout
    .pipeThrough(new TextDecoderStream())
    .pipeThrough(new TextLineStream());

  const results: any[] = [];
  let currentCommit: any = null;
  let commitCount = 0;

  // --- Helper to process a finished commit ---
  const finishCommit = async (commit: any) => {
    if (!commit || commit.changesMap.size === 0) return;

    // 1. Group changes by their exact directory
    // Map< "sprites/group/nested", Map< "path/to/file", "M" > >
    const changesByFolder = new Map<string, Map<string, string>>();

    for (const [path, status] of commit.changesMap) {
      const dir = getDirectory(path);

      if (!changesByFolder.has(dir)) {
        changesByFolder.set(dir, new Map());
      }
      changesByFolder.get(dir)!.set(path, status);
    }

    // 2. Process each folder as a separate Entry
    for (const [folder, folderChanges] of changesByFolder) {
      // Get all existing files strictly inside this folder at this commit
      const allPaths = await getSnapshotFiles(commit.originalSha, folder);

      const mergedFiles = allPaths.map((path) => {
        const status = folderChanges.get(path) || "Existing";
        return {
          path: path,
          status: status,
          url: `${CONFIG.githubUrl}/blob/${commit.originalSha}/${path}`,
        };
      });

      // Filter: Ensure we have files (matches regex)
      if (mergedFiles.length === 0) continue;

      // Sort: Changed first, then alphabetical
      mergedFiles.sort((a, b) => {
        const aChanged = a.status !== "Existing";
        const bChanged = b.status !== "Existing";
        if (aChanged && !bChanged) return -1;
        if (!aChanged && bChanged) return 1;
        return a.path.localeCompare(b.path);
      });

      // Generate ID: <SHA>--<FOLDER-SLUG>
      // e.g. sprites/human/nested -> sprites-human-nested
      const folderSlug = folder.replace(/\//g, "-");
      const uniqueId = `${commit.originalSha}--${folderSlug}`;

      const splitEntry = {
        id: uniqueId,
        sha: commit.originalSha,
        date: commit.date,
        folder: folder,
        message: commit.message,
        author: commit.author,
        files: mergedFiles,
      };

      const changedCount = mergedFiles.filter((f) =>
        f.status !== "Existing"
      ).length;

      // Shorten ID for log
      const logId = uniqueId.length > 50
        ? uniqueId.slice(0, 50) + "..."
        : uniqueId;

      console.log(
        `\r✅ ENTRY: ${logId} | Files: ${mergedFiles.length} (Changed: ${changedCount})`,
      );

      results.push(splitEntry);
    }

    delete commit.changesMap;
  };

  // --- Stream Loop ---
  for await (const line of lines) {
    const trimmed = line.trim();
    if (!trimmed) continue;

    if (trimmed.startsWith("__COMMIT__|")) {
      await finishCommit(currentCommit);

      commitCount++;
      const parts = trimmed.split("|");
      const sha = parts[1];
      const date = parts[2];
      const author = parts[3];
      const message = parts.slice(4).join("|");

      currentCommit = {
        originalSha: sha,
        date,
        author,
        message,
        changesMap: new Map<string, string>(),
      };

      if (commitCount % 100 === 0) {
        await Deno.stdout.write(
          new TextEncoder().encode(`\r⚡ Scanned ${commitCount} commits...`),
        );
      }
      continue;
    }

    if (currentCommit) {
      const parts = trimmed.split("\t");
      const statusRaw = parts[0];
      const status = statusRaw.charAt(0); // A, M
      const filename = parts[1];

      if ((status === "A" || status === "M") && filename) {
        // Matches POSS regex
        if (CONFIG.fileRegex.test(filename)) {
          currentCommit.changesMap.set(filename, statusRaw);
        }
      }
    }
  }

  await finishCommit(currentCommit);

  console.log(`\n\n========================================`);
  console.log(`SCAN COMPLETE`);
  console.log(`Total Git Commits: ${commitCount}`);
  console.log(`Generated Entries: ${results.length}`);
  console.log(`========================================`);

  if (results.length > 0) {
    const jsonOutput = JSON.stringify(results, null, 2);
    await Deno.writeTextFile(CONFIG.outputFile, jsonOutput);
    console.log(`\n💾 Results saved to: ${CONFIG.outputFile}`);
  } else {
    console.log("No matching files found.");
  }
}

main().catch(console.error);


📂 Opening Repo: attic.git
🚀 Git process started. Scanning history...


Promise { <pending> }

✅ ENTRY: 3dac73290304149df980448cc7c98b21a683eb6e--sprites-... | Files: 50 (Changed: 50)
✅ ENTRY: ae45f0a23d0ec4840c72141015411c85fe5bf1e8--horrormo... | Files: 9 (Changed: 9)
✅ ENTRY: ae45f0a23d0ec4840c72141015411c85fe5bf1e8--raymooha... | Files: 49 (Changed: 40)
✅ ENTRY: f8f3370f718c6823ba8441c5f5cdec0a5c16a884--geekmari... | Files: 1 (Changed: 1)
✅ ENTRY: f8f3370f718c6823ba8441c5f5cdec0a5c16a884--raymooha... | Files: 10 (Changed: 10)
✅ ENTRY: 3145154efcc05396dde074d1e62d6fd2cda59340--mc776-sp... | Files: 10 (Changed: 10)
✅ ENTRY: 6a267b6305913507838aafa900b84340c22e26a3--saint_of... | Files: 1 (Changed: 1)
✅ ENTRY: c93121a673d75059e3174c2feade10acb1a75985--saint_of... | Files: 1 (Changed: 1)
✅ ENTRY: b8007c6b534e3c89143de8d3a61977d74eb8774b--saint_of... | Files: 1 (Changed: 1)
✅ ENTRY: b8007c6b534e3c89143de8d3a61977d74eb8774b--saint_of... | Files: 1 (Changed: 1)


SCAN COMPLETE
Total Git Commits: 176
Generated Entries: 10

💾 Results saved to: scan_results-attic.json


In [ ]:
// Run with: deno run --allow-read --allow-write --allow-net --allow-run generate_animations_final_fix.ts

import { ensureDir } from "https://deno.land/std@0.220.0/fs/ensure_dir.ts";
import { join } from "https://deno.land/std@0.220.0/path/mod.ts";

const CONFIG = {
  inputFile: "scan_results-attic.json",
  outputDir: "webp",
  tempDir: "temp_frames",
  delay: 40, // 40 = ~25fps (Doom default)
  concurrency: 5,
};

// FIXED:
// 1. Matches "POSS" at the start of the string OR after a slash (directory separator).
// 2. Matches the Frame char (A-Z) and Angle digit.
// 3. Allows for extra chars in filename (like possa2a8) before the extension.
const SPRITE_REGEX = /(?:^|[\\/])POSS([a-z])(\d).*?\.(png|gif)$/i;

interface FileEntry {
  path: string;
  status: string;
  url: string;
}

interface CommitEntry {
  id: string; // Added ID field
  sha: string;
  date: string;
  files: FileEntry[];
}

async function checkImageMagick() {
  try {
    const cmd = new Deno.Command("magick", { args: ["-version"] });
    const { success } = await cmd.output();
    if (!success) throw new Error();
  } catch {
    console.error("❌ Error: ImageMagick ('magick') not found.");
    Deno.exit(1);
  }
}

// --- Image Utils ---

async function getMaxDimensions(
  folderPath: string,
): Promise<{ w: number; h: number }> {
  const cmd = new Deno.Command("magick", {
    args: ["identify", "-format", "%w,%h\n", join(folderPath, "*")],
  });

  const { stdout } = await cmd.output();
  const output = new TextDecoder().decode(stdout).trim();

  let maxW = 0;
  let maxH = 0;

  for (const line of output.split("\n")) {
    const parts = line.split(",");
    if (parts.length < 2) continue;

    const w = parseInt(parts[0], 10);
    const h = parseInt(parts[1], 10);

    if (!isNaN(w) && w > maxW) maxW = w;
    if (!isNaN(h) && h > maxH) maxH = h;
  }

  return { w: maxW, h: maxH };
}

function getRawUrl(blobUrl: string): string {
  return blobUrl
    .replace("github.com", "raw.githubusercontent.com")
    .replace("/blob/", "/");
}

function isValidImageHeader(data: Uint8Array): boolean {
  if (data.length < 4) return false;
  if (data[0] === 0x47 && data[1] === 0x49 && data[2] === 0x46) return true; // GIF
  if (
    data[0] === 0x89 && data[1] === 0x50 && data[2] === 0x4E && data[3] === 0x47
  ) return true; // PNG
  return false;
}

async function downloadRealImage(
  url: string,
  depth = 0,
): Promise<Uint8Array | null> {
  if (depth > 3) return null;
  const res = await fetch(url);
  if (!res.ok) return null;
  const buffer = await res.arrayBuffer();
  const data = new Uint8Array(buffer);
  if (isValidImageHeader(data)) return data;
  try {
    const textContent = new TextDecoder().decode(data).trim();
    if (textContent.length < 500 && !textContent.includes("\0")) {
      const currentUrlObj = new URL(url);
      const targetUrl = new URL(textContent, new URL(".", currentUrlObj)).href;
      if (targetUrl !== url) return downloadRealImage(targetUrl, depth + 1);
    }
  } catch { /* ignore */ }
  return null;
}

function getAnimationSequence(files: FileEntry[]) {
  const frames = new Map<string, FileEntry>();
  for (const file of files) {
    const match = file.path.match(SPRITE_REGEX);
    if (!match) continue;

    // match[1] is the frame letter (A, B, C...)
    // match[2] is the angle (1-8, 0)
    const letter = match[1].toUpperCase();
    const angle = parseInt(match[2], 10);

    if (frames.has(letter)) {
      const existing = frames.get(letter)!;
      const existingMatch = existing.path.match(SPRITE_REGEX)!;
      const existingAngle = parseInt(existingMatch[2], 10);

      // Prefer Front (1) or Omnidirectional (0) views
      if (existingAngle !== 1 && angle === 1) frames.set(letter, file);
      else if (existingAngle !== 1 && existingAngle !== 0 && angle === 0) {
        frames.set(letter, file);
      }
    } else {
      frames.set(letter, file);
    }
  }
  return Array.from(frames.keys()).sort().map((key) => frames.get(key)!);
}

// --- Main Processing ---

async function processCommit(commit: CommitEntry) {
  // FIXED: Use the JSON 'id' field to ensure uniqueness per folder,
  // preventing race conditions where different folders with same SHA overwrite each other.
  const uniqueId = commit.id || commit.sha;
  const safeId = uniqueId.replace(/[^a-z0-9-]/gi, "_").slice(0, 100);

  const sequence = getAnimationSequence(commit.files);

  if (sequence.length === 0) {
    // console.log(`Skipping ${safeId}: No matching sprite sequence found.`);
    return;
  }

  const commitTempDir = join(CONFIG.tempDir, safeId);
  await ensureDir(commitTempDir);

  let validFramesCount = 0;

  try {
    let index = 0;
    for (const file of sequence) {
      const rawUrl = getRawUrl(file.url);
      const ext = file.path.split(".").pop() || "png";
      const localName = `${String(index).padStart(3, "0")}.${ext}`;
      const localPath = join(commitTempDir, localName);

      const imageBuffer = await downloadRealImage(rawUrl);
      if (!imageBuffer) continue;

      await Deno.writeFile(localPath, imageBuffer);
      validFramesCount++;
      index++;
    }

    if (validFramesCount < 2) return;

    // 1. Calculate Original Max Dimensions
    const { w, h } = await getMaxDimensions(commitTempDir);

    if (w === 0 || h === 0) {
      console.warn(`⚠️  Could not determine dimensions for ${safeId}`);
      return;
    }

    const outputFilename = `poss-${safeId}.webp`;
    const outputPath = join(CONFIG.outputDir, outputFilename);

    // 2. Build ImageMagick Command
    const magickArgs = [
      "-delay",
      String(CONFIG.delay),
      "-dispose",
      "2", // Background clears
      "-background",
      "none",
      "-loop",
      "0",
      join(commitTempDir, "*"),
      // Remove Cyan
      "-transparent",
      "cyan",
      // Align Bottom-Center
      "+repage",
      "-gravity",
      "South",
      "-extent",
      `${w}x${h}`,
      // Scale 2x Hard Edges (Implies interpolation: Nearest)
      "-sample",
      "200%",
      // Force Lossless WebP
      "-define",
      "webp:lossless=true",
      outputPath,
    ];

    const cmd = new Deno.Command("magick", { args: magickArgs });
    const { success, stderr } = await cmd.output();

    if (!success) {
      throw new Error(new TextDecoder().decode(stderr));
    }

    console.log(
      `✅ Generated: ${outputFilename} (${validFramesCount} frames, Size: ${
        w * 2
      }x${h * 2})`,
    );
  } catch (err: any) {
    console.error(`❌ Error processing ${safeId}:`, err.message);
  } finally {
    try {
      await Deno.remove(commitTempDir, { recursive: true });
    } catch { /* ignore */ }
  }
}

async function main() {
  await checkImageMagick();

  console.log(`📂 Reading ${CONFIG.inputFile}...`);
  try {
    const raw = await Deno.readTextFile(CONFIG.inputFile);
    const commits: CommitEntry[] = JSON.parse(raw);

    console.log(`📁 Ensuring output directory: ${CONFIG.outputDir}`);
    await ensureDir(CONFIG.outputDir);
    await ensureDir(CONFIG.tempDir);

    console.log(`🚀 Processing ${commits.length} entries...`);

    for (let i = 0; i < commits.length; i += CONFIG.concurrency) {
      const chunk = commits.slice(i, i + CONFIG.concurrency);
      await Promise.all(chunk.map(processCommit));
    }

    try {
      await Deno.remove(CONFIG.tempDir, { recursive: true });
    } catch { /* ignore */ }

    console.log("\n✨ All Done!");
  } catch (e: any) {
    console.error("Error:", e.message);
  }
}

main().catch(console.error);


Promise { <pending> }

📂 Reading scan_results-attic.json...
📁 Ensuring output directory: webp_x2
🚀 Processing 10 entries...
✅ Generated: poss-f8f3370f718c6823ba8441c5f5cdec0a5c16a884--raymoohawk-sprites-old-gib.webp (9 frames, Size: 114x124)
✅ Generated: poss-ae45f0a23d0ec4840c72141015411c85fe5bf1e8--horrormovierei-sprites-old-zombieman-new-gib.webp (9 frames, Size: 134x122)
✅ Generated: poss-3dac73290304149df980448cc7c98b21a683eb6e--sprites-saint_of_killers-oldposs.webp (21 frames, Size: 114x124)
✅ Generated: poss-ae45f0a23d0ec4840c72141015411c85fe5bf1e8--raymoohawk-sprites-old-zombieman.webp (21 frames, Size: 114x124)
✅ Generated: poss-3145154efcc05396dde074d1e62d6fd2cda59340--mc776-sprites.webp (2 frames, Size: 48x110)

✨ All Done!
